In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install demoji

import re
import demoji
import random
import inflect
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.functional import one_hot
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
# Reading the text
file_path = "/content/drive/MyDrive/human_chat.txt"
with open(file_path, "r") as file:
  lines = file.readlines()

In [ ]:
lines

In [ ]:
len(lines)

1495

In [ ]:
def preprocess_text(text):
        # Remove HTML tags
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text()

        p = inflect.engine() #101
        demoji.download_codes()
        # Remove emojis
        text = demoji.replace(text, "")

        # Remove mentions of "Human 1" and "Human 2"
        text = re.sub(r'\b(?:Human 1|Human 2)\b:?', " ", text)

        # Replace numbers with words
        text = re.sub(r'\b\d+\b', lambda x: p.number_to_words(x.group()), text)

        # Remove special characters, keeping only alphabetic and spaces
        text = re.sub('[^a-zA-Z\s]', ' ', text)

        # Replace specific unicode spaces with standard spaces and trim
        text = text.replace(u'\xa0', u' ').replace('\u200a', ' ').strip()

        return text

In [ ]:
preprocessed_lines = [preprocess_text(line) for line in lines]

In [ ]:
preprocessed_lines[0:5]

In [ ]:
tokenizer = get_tokenizer('basic_english')
tokenized_conv = [tokenizer(conv) for conv in preprocessed_lines]

In [ ]:
tokenized_conv[3]

In [ ]:
features_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_conv,
    min_freq=1,
    specials=['<pad>', '<oov>'],
    special_first=True
)
target_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_conv,
    min_freq=1
)

In [ ]:
features_vocab_total_words = len(features_vocab)
target_vocab_total_words = len(target_vocab)

In [ ]:
print("Features Vocab Length", features_vocab_total_words)
print("Target_vocab_Length", target_vocab_total_words)

In [ ]:
# making ngrams from the conversations
def make_ngrams(tokenized_text):
    list_ngrams = []
    for i in range(1, len(tokenized_text)):
        ngram_sequence = tokenized_text[:i+1]
        list_ngrams.append(ngram_sequence)
    return list_ngrams

In [ ]:
ngrams_list = []
for tokenized_con in tokenized_conv:
    ngrams_list.extend(make_ngrams(tokenized_con))

In [ ]:
# Add Random oov tokens to let the model handle oov tokens
def add_random_oov_tokens(ngram):
    for idx, word in enumerate(ngram[:-1]):
        if random.uniform(0, 1) < 0.1:
            ngram[idx] = '<oov>'
    return ngram

In [ ]:
ngrams_list_oov = []
for ngram in ngrams_list:
    ngrams_list_oov.append(add_random_oov_tokens(ngram))
print(any('<oov>' in ngram for ngram in ngrams_list_oov))

In [ ]:
def text_to_numerical_sequence(tokenized_text):
    tokens_list = []
    if tokenized_text[-1] in target_vocab.get_itos():
        for token in tokenized_text[:-1]:
            num_token = features_vocab[token] if token in features_vocab.get_itos() else features_vocab['<oov>']
            tokens_list.append(num_token)
        num_token = target_vocab[tokenized_text[-1]]
        tokens_list.append(num_token)
        return tokens_list
    return None

In [ ]:
input_sequences = [text_to_numerical_sequence(sequence) for sequence in ngrams_list_oov if text_to_numerical_sequence(sequence)]

In [ ]:
X = [sequence[:-1] for sequence in input_sequences]
y = [sequence[-1] for sequence in input_sequences]
len(X[0]), y[0]

In [ ]:
print(X[3],y[3])

In [ ]:
longest_sequence_feature = max(len(sequence) for sequence in X)
longest_sequence_feature

In [ ]:
padded_X = [F.pad(torch.tensor(sequence), (longest_sequence_feature - len(sequence),0), value=0) for sequence in X]
padded_X[1], X[2], len(padded_X[1])

In [ ]:
padded_X = torch.stack(padded_X)
y = torch.tensor(y)
type(y), type(padded_X)

In [ ]:
y_one_hot = one_hot(y, num_classes=target_vocab_total_words)

In [ ]:
data = TensorDataset(padded_X, y_one_hot)

In [ ]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
batch_size = 32

In [ ]:
train_data, test_data = random_split(data, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
class My_BiLSTM(nn.Module):
    def __init__(self, features_vocab_total_words, target_vocab_total_words, embedding_dim, hidden_dim):
        super(My_BiLSTM, self).__init__()
        self.embedding = nn.Embedding(features_vocab_total_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, target_vocab_total_words)

    def forward(self, x):
        x = x.to(self.embedding.weight.device)
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        # Since the LSTM is bidirectional, we concatenate the last hidden state of the forward direction
        # and the first hidden state of the backward direction before passing it to the fully connected layer
        # For batch_first=True, the last timestep of the forward direction is lstm_out[:, -1, :hidden_dim]
        # and the first timestep of the backward direction is lstm_out[:, 0, hidden_dim:]
        output = self.fc(torch.cat((lstm_out[:, -1, :hidden_dim], lstm_out[:, 0, hidden_dim:]), dim=1))
        return output

In [ ]:
features_vocab_total_words = 2749
target_vocab_total_words = 2747
embedding_dim = 128
hidden_dim = 200
epochs = 50

In [ ]:
model = My_BiLSTM(features_vocab_total_words, target_vocab_total_words, embedding_dim=embedding_dim, hidden_dim=hidden_dim)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0009)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
def calculate_topk_accuracy(model, data_loader, k=3):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Forward pass
            output = model(batch_x)

            # Get top-k predictions
            _, predicted_indices = output.topk(k, dim=1)

            # Check if the correct label is in the top-k predictions
            correct_predictions += torch.any(predicted_indices == torch.argmax(batch_y, dim=1, keepdim=True), dim=1).sum().item()
            total_predictions += batch_y.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy

In [ ]:
all_accuracies = []
all_losses = []
for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.argmax(dim=1))
        loss.backward()
        optimizer.step()

    if epoch % 5 == 0:
        accuracy = calculate_topk_accuracy(model, train_loader)
        print(f'Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}, Train K-Accuracy: {accuracy * 100:.2f}%')
        all_accuracies.append(accuracy)
        all_losses.append(loss.item())

In [ ]:
epoch_list = [i for i in range(1,epochs,5)]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

axes[0].plot(epoch_list, all_accuracies, color='#5a7da9', label='Accuracy', linewidth=3)
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy Graph')
axes[0].grid(True)

axes[1].plot(epoch_list, all_losses, color='#adad3b', label='Accuracy', linewidth=3)
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Loss')
axes[1].set_title('Loss Graph')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
accuracy = calculate_topk_accuracy(model, test_loader)
print(f'Test K-Accuracy: {accuracy * 100:.2f}%')

In [ ]:
model_path = '/content/drive/MyDrive/CampusX_Lecture/Day_3/my_bilstm_model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

In [ ]:
model_path = '/content/drive/MyDrive/CampusX_Lecture/Day_3/my_bilstm_model.pth'
torch.save(model, model_path)
print(f'Model saved to {model_path}')

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, features_vocab_total_words, target_vocab_total_words, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(features_vocab_total_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, target_vocab_total_words)

    def forward(self, x):
        x = x.to(self.embedding.weight.device)
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        # Since the LSTM is bidirectional, we concatenate the last hidden state of the forward direction
        # and the first hidden state of the backward direction before passing it to the fully connected layer
        # For batch_first=True, the last timestep of the forward direction is lstm_out[:, -1, :hidden_dim]
        # and the first timestep of the backward direction is lstm_out[:, 0, hidden_dim:]
        output = self.fc(torch.cat((lstm_out[:, -1, :hidden_dim], lstm_out[:, 0, hidden_dim:]), dim=1))
        return output

In [ ]:
features_vocab_total_words = 2749
target_vocab_total_words = 2747
embedding_dim = 128
hidden_dim = 200
saved_model = BiLSTM(features_vocab_total_words, target_vocab_total_words, embedding_dim, hidden_dim)

In [ ]:
model_path = '/content/drive/MyDrive/CampusX_Lecture/Day_3/my_bilstm_model.pth'
saved_model.load_state_dict(torch.load(model_path))

In [ ]:
def text_to_numerical_sequence_test(tokenized_text):
    tokens_list = []
    for token in tokenized_text:
        num_token = features_vocab[token] if token in features_vocab.get_itos() else features_vocab['<oov>']
        tokens_list.append(num_token)
    return tokens_list

In [ ]:
def use_model(input_list, top_k=5):
    saved_model.eval()
    output_list = []
    for data in input_list:
        sentence = data[0]
        num_words = data[1]
        for i in range(num_words):
            output_of_model = []
            tokenized_input_test = tokenizer(sentence)
            tokenized_sequence_input_test = text_to_numerical_sequence_test(tokenized_input_test)
            padded_tokenized_sequence_input_test = F.pad(torch.tensor(tokenized_sequence_input_test),
                                                         (longest_sequence_feature - len(tokenized_sequence_input_test)-1, 0),
                                                         value=0)
            output_test_walking = model(padded_tokenized_sequence_input_test.unsqueeze(0))
            # Get the top k predictions and their indices
            top_predictions = torch.topk(output_test_walking, top_k)
            # Iterate over the top predictions
            for index in top_predictions.indices.squeeze():
                # Lookup the token for each index and append it to the output sentence
                sentence_with_predictions = sentence + ' ' + target_vocab.lookup_token(index.item())
                output_list.append(sentence_with_predictions)
        # Update the input sentence with the last predicted word
        sentence = output_list[-1]
    return output_list

In [ ]:
input_test = [['Hi', 1], ['Hello', 1]]

In [ ]:
outputs_model = use_model(input_test,5)
outputs_model

In [ ]:
class TextProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.p = inflect.engine()
        demoji.download_codes()

    def read_file(self):
        with open(self.file_path, "r") as file:
            return file.readlines()

    def preprocess_text(self, text):
        # Remove HTML tags
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text()

        # Remove emojis
        text = demoji.replace(text, "")

        # Remove mentions of "Human 1" and "Human 2"
        text = re.sub(r'\b(?:Human 1|Human 2)\b:?', " ", text)

        # Replace numbers with words
        text = re.sub(r'\b\d+\b', lambda x: self.p.number_to_words(x.group()), text)

        # Remove special characters, keeping only alphabetic and spaces
        text = re.sub('[^a-zA-Z\s]', ' ', text)

        # Replace specific unicode spaces with standard spaces and trim
        text = text.replace(u'\xa0', u' ').replace('\u200a', ' ').strip()

        return text

    def preprocess_lines(self):
        lines = self.read_file()
        preprocessed_lines = [self.preprocess_text(line) for line in lines]
        return preprocessed_lines

In [ ]:
file_path = "/content/drive/MyDrive/human_chat.txt"

processor = TextProcessor(file_path)
preprocessed_lines = processor.preprocess_lines()

In [ ]:
class TextTokenizerAndVectorizer:
    def __init__(self, conversations, oov_rate=0.1, specials=['<pad>', '<oov>']):
        self.tokenizer = get_tokenizer('basic_english')
        self.conversations = conversations
        self.oov_rate = oov_rate
        self.specials = specials
        self.tokenized_text = [self.tokenizer(conv) for conv in conversations]
        self.features_vocab = self.build_vocab(special_first=True)
        self.target_vocab = self.build_vocab(special_first=False)
        self.num_classes = len(self.target_vocab)

    def build_vocab(self, special_first=True):
        specials = self.specials if special_first else []
        return build_vocab_from_iterator(
            self.tokenized_text,
            min_freq=1,
            specials=specials,
            special_first=special_first
        )

    def make_ngrams(self, text):
        ngrams = []
        for i in range(1, len(text)):
            ngram_sequence = text[:i+1]
            ngrams.append(ngram_sequence)
        return ngrams

    def add_random_oov_tokens(self, ngram):
        for idx, word in enumerate(ngram[:-1]):
            if random.uniform(0, 1) < self.oov_rate:
                ngram[idx] = '<oov>'
        return ngram

    def text_to_numerical_sequence(self, tokenized_text):
        tokens_list = []
        if tokenized_text[-1] in self.target_vocab.get_itos():
            for token in tokenized_text[:-1]:
                num_token = self.features_vocab[token] if token in self.features_vocab.get_itos() else self.features_vocab['<oov>']
                tokens_list.append(num_token)
            num_token = self.target_vocab[tokenized_text[-1]]
            tokens_list.append(num_token)
            return tokens_list
        return None

    def process_text(self):
        ngrams_list = [ngram for text in self.tokenized_text for ngram in self.make_ngrams(text)]
        ngrams_list_oov = [self.add_random_oov_tokens(ngram) for ngram in ngrams_list]
        input_sequences = [self.text_to_numerical_sequence(seq) for seq in ngrams_list_oov if self.text_to_numerical_sequence(seq)]

        X = [seq[:-1] for seq in input_sequences]
        y = [seq[-1] for seq in input_sequences]
        longest_sequence_feature = max(len(seq) for seq in X)

        padded_X = torch.stack([F.pad(torch.tensor(seq), (longest_sequence_feature - len(seq),0), value=self.features_vocab['<pad>']) for seq in X])
        y = torch.tensor(y)
        y_one_hot = one_hot(y, num_classes=self.num_classes)

        return padded_X, y_one_hot, len(self.features_vocab), len(self.target_vocab)

In [ ]:
txt_processor = TextTokenizerAndVectorizer(preprocessed_lines)
X, y, features_vocab_total_words, target_vocab_total_words = txt_processor.process_text()

In [ ]:
# Assuming X and y are your features and targets tensors respectively
data = TensorDataset(X, y)

# Calculate split sizes
total_size = len(data)
train_size = int(0.70 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size  # Ensure the sum of splits equals total_size

# Split the dataset
train_data, val_data, test_data = random_split(data, [train_size, val_size, test_size])

# Define batch size
batch_size = 32

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
class My_BiLSTM(nn.Module):
    def __init__(self, features_vocab_total_words, target_vocab_total_words, embedding_dim, hidden_dim):
        super(My_BiLSTM, self).__init__()
        self.embedding = nn.Embedding(features_vocab_total_words, embedding_dim)
        # Specify bidirectional=True to use a BiLSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        # Since it's bidirectional, concatenate the hidden states from both directions
        self.fc = nn.Linear(hidden_dim * 2, target_vocab_total_words)

    def forward(self, x):
        x = x.to(self.embedding.weight.device)
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        # Since the LSTM is bidirectional, we concatenate the last hidden state of the forward direction
        # and the first hidden state of the backward direction before passing it to the fully connected layer
        # For batch_first=True, the last timestep of the forward direction is lstm_out[:, -1, :hidden_dim]
        # and the first timestep of the backward direction is lstm_out[:, 0, hidden_dim:]
        output = self.fc(torch.cat((lstm_out[:, -1, :hidden_dim], lstm_out[:, 0, hidden_dim:]), dim=1))
        return output

In [ ]:
embedding_dim = 128
hidden_dim = 256
epochs = 50
model = My_BiLSTM(features_vocab_total_words, target_vocab_total_words, embedding_dim=embedding_dim, hidden_dim=hidden_dim)

In [ ]:
class BiLSTMTrainer:
    def __init__(self, model, train_loader, val_loader=None, epochs=50, lr=0.0009, device=None, k=3, patience=5):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.epochs = epochs
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.best_val_loss = float('inf')
        self.k = k
        self.patience = patience

    def calculate_topk_accuracy(self, data_loader):
        self.model.eval()
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for batch_x, batch_y in data_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                output = self.model(batch_x)
                _, predicted_indices = output.topk(self.k, dim=1)
                correct_predictions += torch.any(predicted_indices == torch.argmax(batch_y, dim=1, keepdim=True), dim=1).sum().item()
                total_predictions += batch_y.size(0)

        accuracy = correct_predictions / total_predictions
        return accuracy

    def train(self):

        train_accuracies = []
        val_accuracies = []
        train_losses = []
        val_losses = []
        count = 0

        for epoch in range(self.epochs):
            self.model.train()
            epoch_losses = []
            for batch_X, batch_y in self.train_loader:
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y.argmax(dim=1))
                loss.backward()
                self.optimizer.step()
                epoch_losses.append(loss.item())

            avg_loss = sum(epoch_losses) / len(epoch_losses)
            train_losses.append(avg_loss)

            if self.val_loader:
                val_loss = self.validate()
                val_losses.append(val_loss)
                if val_loss < self.best_val_loss:
                    self.best_val_loss = val_loss
                    torch.save(model.state_dict(), 'best_model.pth')
                    print(f'new best validation loss: {val_loss:.4f}')
                    count = 0
                else:
                    count += 1
                    if count > self.patience:
                      print(f'Early stopping at epoch {epoch + 1}')
                      break  # Early stopping condition

            train_accuracy = self.calculate_topk_accuracy(self.train_loader)
            val_accuracy = self.calculate_topk_accuracy(self.val_loader)

            train_accuracies.append(train_accuracy)
            val_accuracies.append(val_accuracy)

            if epoch % 5 == 0:
                print(f'Epoch {epoch}/{self.epochs} ||Train Loss: {avg_loss:.4f} || Train K-Accuracy: {train_accuracy * 100:.2f}% || Val Loss :{val_loss:.4f} || Validation K-Accuracy: {val_accuracy * 100:.2f}%')

        self.plot_results(train_accuracies, train_losses, val_accuracies, val_losses)

    def validate(self):
        self.model.eval()
        val_losses = []
        with torch.no_grad():
            for batch_x, batch_y in self.val_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y.argmax(dim=1))
                val_losses.append(loss.item())
        return sum(val_losses) / len(val_losses)

    def plot_results(self, train_accuracies, train_losses, val_accuracies, val_losses):

        epochs_x = list(range(1, len(train_accuracies) + 1))
        epochs_x_ = list(range(1, len(train_losses) + 1))
        plt.figure(figsize=(12, 6))

        # Creating two y-axes
        fig, ax1 = plt.subplots()

        # Plotting training and validation accuracies
        plt.subplot(1, 2, 1)
        plt.plot(epochs_x, train_accuracies, 'b-o', label='Train Accuracy')
        plt.plot(epochs_x, val_accuracies, 'r-+', label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend(loc='best')

        # Plotting training and validation losses
        plt.subplot(1, 2, 2)
        plt.plot(epochs_x_, train_losses, 'g-o', label='Train Loss')
        plt.plot(epochs_x_, val_losses, 'k-+', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend(loc='best')

        plt.tight_layout()
        plt.show()

In [ ]:
trainer = BiLSTMTrainer(model, train_loader=train_loader, val_loader=val_loader)

In [ ]:
trainer.train()